# Description
In this notebook we export the parameter estimates as a DataFrame and a LaTeX table.

In [1]:
import arviz
import json
import pandas
import pathlib


DP_PROCESSED = pathlib.Path("processed")

### Read models and dataset

In [2]:
with open(DP_PROCESSED / "full_dataset_mle.json") as jfile:
    theta_dict = json.load(jfile)
idata = arviz.from_netcdf(DP_PROCESSED / "full_posterior.nc")
idata

Inference data with groups:
	> posterior
	> sample_stats
	> observed_data

Warmup iterations saved (warmup_*).

In [3]:
arviz.ess(idata)

<xarray.Dataset>
Dimensions:    (replicate: 28)
Coordinates:
  * replicate  (replicate) object 'A02' 'A03' 'A04' 'A05' ... 'D06' 'D07' 'D08'
Data variables:
    X0_mu      float64 8.076e+03
    F_offset   (replicate) float64 8.165e+03 8.191e+03 ... 8.079e+03 8.119e+03
    X0         (replicate) float64 7.94e+03 7.636e+03 ... 8.893e+03 8.576e+03
    S0         float64 8.63e+03
    Y_XS       float64 8.616e+03
    mu_max     float64 8.461e+03
    K_S        float64 5.338e+03

### Create Dataframe and Save it

In [4]:
df = arviz.summary(idata, var_names="S0,mu_max,Y_XS,K_S,X0_mu,X0".split(","), hdi_prob=0.9, round_to="none")
rename_dict = {}
for k in df.index.values:
    if "[" in k:
        rvname, cindex = k.strip("]").split("[")
        cindex = int(cindex)
        cname = tuple(idata.posterior[rvname].coords.keys())[2+0]
        cval = idata.posterior.coords[cname].values[cindex]
        rename_dict[k] = f"{rvname}_{cval}"
df = df.rename(index=rename_dict)
df = df.assign(MLE=pandas.Series(theta_dict)).fillna("-")
cols = "MLE,mean,sd,hdi_5%,hdi_95%,r_hat"#,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail"
df = df[cols.split(",")]
df

,MLE,mean,sd,hdi_5%,hdi_95%,r_hat
S0,17.048861,15.219965,0.102652,1.505286e+01,15.389549,1.000217
mu_max,0.424605,0.423888,0.000744,4.226556e-01,0.425101,1.000432
Y_XS,0.667471,0.747687,0.005071,7.393388e-01,0.755953,1.000208
K_S,0.001005,0.001741,0.001680,4.390320e-07,0.003945,1.003217
X0_mu,-,0.269741,0.009849,2.534624e-01,0.285827,1.000602
X0_A02,0.246174,0.266939,0.052755,1.805123e-01,0.349428,1.000127
X0_A03,0.265208,0.341763,0.022076,3.054174e-01,0.378070,1.001492
X0_A04,0.279382,0.299998,0.006416,2.894118e-01,0.310431,1.000118
X0_A05,0.266249,0.268150,0.002306,2.643187e-01,0.271903,1.000106
X0_A06,0.249197,0.250789,0.001619,2.481223e-01,0.253455,1.000371


In [5]:
print(df.to_latex(bold_rows=True, longtable=False, label="tab_parameter_estimates"))

\begin{table}
\centering
\label{tab_parameter_estimates}
\begin{tabular}{llrrrrr}
\toprule
{} &        MLE &       mean &        sd &        hdi\_5\% &    hdi\_95\% &     r\_hat \\
\midrule
\textbf{S0    } &  17.048861 &  15.219965 &  0.102652 &  1.505286e+01 &  15.389549 &  1.000217 \\
\textbf{mu\_max} &   0.424605 &   0.423888 &  0.000744 &  4.226556e-01 &   0.425101 &  1.000432 \\
\textbf{Y\_XS  } &   0.667471 &   0.747687 &  0.005071 &  7.393388e-01 &   0.755953 &  1.000208 \\
\textbf{K\_S   } &   0.001005 &   0.001741 &  0.001680 &  4.390320e-07 &   0.003945 &  1.003217 \\
\textbf{X0\_mu } &          - &   0.269741 &  0.009849 &  2.534624e-01 &   0.285827 &  1.000602 \\
\textbf{X0\_A02} &   0.246174 &   0.266939 &  0.052755 &  1.805123e-01 &   0.349428 &  1.000127 \\
\textbf{X0\_A03} &   0.265208 &   0.341763 &  0.022076 &  3.054174e-01 &   0.378070 &  1.001492 \\
\textbf{X0\_A04} &   0.279382 &   0.299998 &  0.006416 &  2.894118e-01 &   0.310431 &  1.000118 \\
\textbf{X0\_A05} & 

In [6]:
%load_ext watermark
%watermark

Last updated: 2021-12-16T00:27:47.933320+01:00

Python implementation: CPython
Python version       : 3.7.9
IPython version      : 7.24.1

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 158 Stepping 10, GenuineIntel
CPU cores   : 6
Architecture: 64bit

